# Do people with physical disabilities have disadvantages in education and work?

In [17]:
# import library and dataset

import pandas as pd

df = pd.read_csv('./data/stack-overflow-developer-survey-2021/2021 Stack Overflow Survey Responses.csv', encoding="ISO-8859-1")

df.head()

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,NaN,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,62268.0
1,2,I am a student who is learning to code,"Student, full-time",Netherlands,NaN,NaN,"Bachelorâs degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",7,...,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN
2,3,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,NaN,NaN,"Bachelorâs degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",NaN,...,18-24 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,None of the above,Appropriate in length,Easy,NaN
3,4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Masterâs degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,NaN,NaN,...,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,NaN,Appropriate in length,Neither easy nor difficult,NaN
4,5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Masterâs degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,Friend or family member,17,...,25-34 years old,Man,No,NaN,White or of European descent,None of the above,NaN,Appropriate in length,Easy,NaN


In [18]:
# display all given unique answers regarding accessibility
# multiple answers were possible

df.Accessibility.unique()

array(['None of the above', 'I am deaf / hard of hearing', nan,
       'Prefer not to say', 'I am blind / have difficulty seeing',
       'I am deaf / hard of hearing;I am blind / have difficulty seeing;I am unable to / find it difficult to type;I am unable to / find it difficult to walk or stand without assistance',
       'Or, in your own words:',
       'I am unable to / find it difficult to walk or stand without assistance',
       'I am blind / have difficulty seeing;Or, in your own words:',
       'I am blind / have difficulty seeing;I am unable to / find it difficult to type',
       'I am deaf / hard of hearing;I am blind / have difficulty seeing;I am unable to / find it difficult to type;I am unable to / find it difficult to walk or stand without assistance;Or, in your own words:',
       'I am unable to / find it difficult to type',
       'I am blind / have difficulty seeing;I am unable to / find it difficult to type;I am unable to / find it difficult to walk or stand withou

In [19]:
# divide dataset in survey participants with and without limitation

df_temp = df.dropna(subset=['Accessibility']) # filter out Null values
df_temp = df_temp[~df_temp.Accessibility.isin(['Or, in your own words:', 'Prefer not to say'])] # filter out values we cannot work with

df_acc = df_temp[df_temp.Accessibility != 'None of the above'] # Dataset with participants with limitation
df_not_acc = df_temp[df_temp.Accessibility == 'None of the above'] # Dataset with participants without limitation

In [20]:
print('Number Of Survey Participants With Physical Limitation:', str(df_acc.shape[0])+',', str(round(df_acc.shape[0]/(df_acc.shape[0]+df_not_acc.shape[0]),2))+'%,', '\nNumber Of Survey Participants Without Physical Limitation:', str(df_not_acc.shape[0])+',', str(round(df_not_acc.shape[0]/(df_acc.shape[0]+df_not_acc.shape[0]),2))+'%')

Number Of Survey Participants With Physical Limitation: 2118, 0.03%, 
Number Of Survey Participants Without Physical Limitation: 72725, 0.97%


In [21]:
# calculate the mean compensation for both groups 

mean_acc = round(df_acc.ConvertedCompYearly.mean(), 2)
mean_not_acc = round(df_not_acc.ConvertedCompYearly.mean(), 2)
 
print('Mean Salary Of Survey Participants With Physical Limitation:', mean_acc, '\nMean Salary Of Survey Participants Without Physical Limitation:', mean_not_acc, '\nDifference:', round(mean_acc-mean_not_acc, 2))

Mean Salary Of Survey Participants With Physical Limitation: 219832.56 
Mean Salary Of Survey Participants Without Physical Limitation: 115596.82 
Difference: 104235.74


In [22]:
# visualize the differences in degree of education of both groups

def clean(df):
    # calculates the distribution of the degrees of education in the dataset
    ed_dist = df['EdLevel'].value_counts(normalize=True).reset_index()
    ed_dist.rename(columns={'index': 'EdLevel', 'EdLevel': 'count'}, inplace=True)
    ed_dist.set_index('EdLevel', inplace=True)
    return ed_dist

mean_acc_perc = clean(df_acc) # get the distribution of the degrees of education of participants with limitations
mean_not_acc_perc = clean(df_not_acc) # get the distribution of the degrees of education of participants without limitations

comp_df = pd.merge(mean_acc_perc, mean_not_acc_perc, left_index=True, right_index=True) # merge both datasets
comp_df.columns = ['Physically Limited', 'Not Physically Limited'] # rename columns
comp_df['Diff_Ed_Values'] = comp_df['Physically Limited'] - comp_df['Not Physically Limited'] # calculate the differences
comp_df.style.bar(subset=['Diff_Ed_Values'], align='mid', color=['#d65f5f', '#5fba7d']) # plot the columns and the differences

,Physically Limited,Not Physically Limited,Diff_Ed_Values
EdLevel,,,
"Bachelorâs degree (B.A., B.S., B.Eng., etc.)",0.322428,0.430199,-0.107771
Some college/university study without earning a degree,0.170697,0.126753,0.043944
"Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",0.158369,0.114095,0.044274
"Masterâs degree (M.A., M.S., M.Eng., MBA, etc.)",0.138928,0.213564,-0.074635
Primary/elementary school,0.073020,0.027711,0.045309
"Associate degree (A.A., A.S., etc.)",0.043148,0.026389,0.016759
Something else,0.041726,0.016156,0.025570
"Other doctoral degree (Ph.D., Ed.D., etc.)",0.027027,0.031017,-0.003990
"Professional degree (JD, MD, etc.)",0.024656,0.014117,0.010539
